## Ejercicio 7

Mozilla Common Voice es un proyecto (https://commonvoice.mozilla.org/es/datasets) que desarrolla una base de datos de voces, abierta y multi idioma, que cualquiera puede usar para entrenar aplicaciones que utilicen la voz como interfaz.

En particular cuenta con un pequeño corpus denominado **Single Word** que contiene las palabras habladas SI, NO, Hey, Firefox y dígitos del cero al nueve.


<div style="text-align: center;">
    <img src='../../images/p5-ej7.png' width="60%">
</div>

### a)

Utilizando los ejemplos de una versión seleccionada de estos audios en el Moodle del curso,
transformarlos en imágenes para que puedan ser procesados por la red siguiendo los siguientes
criterios:
- Eliminar los silencios iniciales y finales. La comparación de los silencios en 2 audios produce una falsa coincidencia.
- Elegir un tamaño fijo para los audios. Todas las imágenes deben tener el mismo tamaño. Como las palabras del corpus son breves un tamaño de 0.75 segundos es razonable.
- Achicar/agrandar los audios manteniendo el tono de la voz
- Pasar el audio del dominio del tiempo al domino de la frecuencia. Dividir el audio en pequeños intervalos y obtener un conjunto de valores que determinen las frecuencias presentes. Para esto puede utilizarse la transformada de Fourier (FFT) o los coeficientes cepstrales en frecuencia mel (MFCC) que aproximan la sensibilidad del oído humano.
- Finalmente convertir los coeficientes en una imagen con compresión sin pérdida como PNG.

### b)

Utilizando las imágenes generadas en a), entrenar un modelo de red neuronal convolucional que permita reconocer las palabras del corpus Single Word